# Week #6 - The finite element method - Static problem

It is one of the oldest numerical methods. At the beginning people
were interested in using it for describing the movement and forces
of buildings.

The idea is, for example, consider a building that one is interested
in analyzing and divide it into several bins. Right after, consider
all the beams which might be exercising force on it. These connections
between the bin which is being analyzed and those at its surrounding
is called *assembly*. Mathematically, this approach gives us linear
system equations to be solved. According professor Igel, it was the
first method which allowed us to solve very complex geometries. Like
vulcanos, vibrations on a wing of a bird and other different stuff!

## Math of the finite-element method

Consider the 1D elastic wave equation

$$
    \rho(x)\partial_t^2u(x,t) = \partial_x\big[\mu(x)\partial_xu(x,t)\big] + f(x,t)\,.
$$

If we start by the static case, the LHS is zero. If we go further in the approximations,
considering a homogenous medium, the shear modulus is constant in space and we obtain the
Poisson equation

$$
    -\mu\partial_x^2 u(x) = f(x)\,.
$$

## The weak form and the Garlekin principle

The Poisson equation, written as above, is called the Strong form of the equation.
The weak form is when the solution is not absolutely correc, but it is only valid
within the space of the basis/test functions. *Need to understand better what he means
by test function*

Let's assume that we have a *basis function* $v(x)$. If we multiply the Poisson equation
by it and integrate over the domain $D$, we have

$$
    -\int_D\mathrm{d}x\mu\partial_x^2 u(x)v(x) = \int_D\mathrm{d}xf(x)v(x)\,,
$$
integrating it by parts, we obtain

$$
    -\int_D\mathrm{d}x\mu\partial_x^2 u(x)v(x) = -\left[\mu v(x)\partial_xu(x)\right]_D +
        \mu\int_D\mathrm{d}x\partial_xv(x)\partial_xu(x)\,.
$$
A great advantage of this technique is that the boundary condition can be implemented
directly by the first term on the RHS. In seismology, it is pretty common to have
$\partial_xu(x)\rvert_D\equiv 0$, which tells us that there is no change in the pressure
amplitudes at the boundaries. Consequently, the equation in its weak form is given by

$$
    \int_D\mathrm{d}x\mu \partial_x v(x)\partial_xu(x) = \int_D\mathrm{d}xf(x) v(x)
$$

> The equation above is defined within the continuous world. Our next step is to move
towards the discrete world.

### Discretization

The discretization is given by

$$
u(x) \simeq \bar{u}(x) \equiv \sum_{i=1}^N u_i\phi_i(x)\,,
$$
replacing it into the weak Poisson equation, we ahve

$$
    \mu\int_D\mathrm{d}x\partial_xv(x)\partial_x\bar{u}(x) = \int_D\mathrm{d}xf(x) v(x)\,,
$$
where we have assumed that the shear modulus is constant.

**Most important step**

Now we give a more explicit form for the test function, we assume that
$v \equiv \phi_j$. Then, the equation above reads

$$
    \sum_{i=1}^Nu_i\mu\int_D\mathrm{d}x\partial_x\phi_j\partial_x\phi_i = \int_D\mathrm{d}xf(x) \phi_j\,,
$$


The equation above is the one which will allow us to solve the problem by handling
linear equations. It is known as Garlekin principle, which was invented by Garlekin
as a solution strategy for such finite element type problem.

## Solution scheme

The question that we pose ourselves to answer is: *which kind of 
basis functions shall we employ in the finite element method?*
The classical method is to approximate the functions inside
the elements using linear approximations to it. Therefore,
divide our space into segments and therein each segment
we interpolate the two boundary element points by a straight
line. A possible way to achieve this is by using cardinal functions,
*i.e.,* functions which are zero everywhere except at one point.
Doing so in our discrete scheme, and interpolating those points,
we get a function which looks like a hat

\begin{equation}
\cdots -o-o-/^O\-o-o-\cdots
\end{equation}

\begin{align}
    \phi_i &= \frac{x-x_{i-1}}{x_i-x_{i-1}}\,, \text{for }x_{i-1} \leq x \leq x_i\\
           &= \frac{x_{i+1}-x}{x_{i+1}-x_{i}}\,, \text{for }x_{i} \leq x \leq x_{i+1}\\
           &= 0\,, \text{elsewhere}
\end{align}


One of the huge differences between the finite difference method that we have studied
earlier is that it allowed us to know the field only **at the grid points**, whereas
the finite element method interpolate those points allowing us to infer the value
fields **in between the grid points**. Therefore the finite element method allows
us to know the field in the entire domain.

If we define

$$
    K_{ij} \equiv \mu\int_D\mathrm{d}x\partial_x\phi_i\partial_x\phi_j\,,
$$
we have a matrix equation for the problem

$$
\sum_{i=1}^Nu_iK_{ij} = f_j\, \rightarrow \mathbf{K}^\mathrm{T}\cdot\mathbf{u} = \mathbf{f}\,.
$$

By the description above, it is rather straightforward to see that since $\mathbf{f}$ has
$N$ componentes, one has to solve $N$-linear equations.

## Boundary conditions

An easy way to implement the proper boundary conditions is splitting it from
the "bulk" of the vector, that is

$$
    \sum_{i=1}^Nu_i\phi_i = \sum_{i=2}^{N-1}u_i\phi_i + u_1\phi_1 + u_N\phi_N\,.
$$
Employing it into the field equation, we have

$$
    \sum_{i=2}^{N-1}u_i\mu\int_D\mathrm{d}x\partial_x\phi_j\partial_x\phi_i = \int_D\mathrm{d}xf(x) \phi_j
        + u_1\mu\int_D\mathrm{d}x\partial_x\phi_j\partial_x\phi_1
        + u_N\mu\int_D\mathrm{d}x\partial_x\phi_j\partial_x\phi_N\,,
$$
where $u_1$ and $u_N$ corresponds to $u(x_\text{min})$ and $u(x_\text{max})$, respectively.
Consequently, we see that in some sense the boundaries act like a force term.

### The stifness matrix $K_{ij}$
Now, one of the fundamental points, we transform the coordinate used ($x$, for instance),
into a local coordinate system, defined in between grid points. We defined a new
variable $\xi$ and the size of element by $h_i$
$$
    \xi \equiv x-x_i\,,\quad h_i \equiv x_i - x_{i-1}\,.
$$

Consequently, the basis functions as defined previously now are written
in a simpler form as

\begin{align}
    \phi_i(\xi) &= \frac{\xi}{h_i} + 1\,, \text{for }-h_i \leq \xi \leq 0\\
           &= 1-\frac{\xi}{h_i}\,, \text{for }0 \leq \xi \leq h_i\\
           &= 0\,, \text{elsewhere}\,.
\end{align}
We can also now compute the derivatives with respect to the new variables
(defined in between the grid points, *i.e.,* therein the elements, to obtain

\begin{align}
    \partial_\xi\phi_i(\xi) &= \frac{1}{h_i}\,, \text{for }-h_i \leq \xi \leq 0\\
           &=-\frac{1}{h_i}\,, \text{for }0 \leq \xi \leq h_i\\
           &= 0\,, \text{elsewhere}\,.
\end{align}
Therefore, if we now plug the above results into the stifness matrix elements,
you obtain

$$
    K_{ij} = \mu\int_{D_\xi}\mathrm{d}\xi\partial_\xi\phi_i(\xi)\partial_\xi\phi_j(\xi)
$$

If we consider first the element $K_{11}$ we have

$$
    K_{11} = \mu\int_0^h\mathrm{d}\xi \left(-\frac{1}{h}\right)\left(-\frac{1}{h}\right) = \frac{\mu}{h}\,.
$$
The element $K_{22}$ on the other hand, has the entire cycle, therefore:

$$
    K_{22} = \mu\int_{-h}^0\mathrm{d}\xi \left(\frac{1}{h}\right)\left(\frac{1}{h}\right) + 
        \mu\int_0^h\mathrm{d}\xi \left(-\frac{1}{h}\right)\left(-\frac{1}{h}\right) = \frac{2\mu}{h}
$$

An off diagonal non-vanishing element is $K_{12}$. The derivative $\partial_\xi\phi_1$
is negative at this part, whereas the $\partial_\xi\phi_2$ is positive. Furthermore,
within the domain where $\partial_\xi\phi_2$ is negative, $\partial_\xi\phi_1$ is
zero. Consequently we have only one region to integrate, differently than what
happens in $K_{22}$. Therefore

$$
    K_{12} = \mu\int_0^h\mathrm{d}\xi \left(-\frac{1}{h}\right)\left(\frac{1}{h}\right) = -\frac{\mu}{h}
$$

Therefore, it is rather straightforward to see that the stiffness matrix is $\mathbf{K}$
is given by
$$
\mathbf{K} = 
\begin{bmatrix}
1 & -1 & 0 & 0 & \cdots & 0\\
-1 & 2 & -1 & 0 &\cdots & 0\\
0 & -1 & 2 & -1 & \cdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\
\vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\
0 & \cdots  & 0 & \cdots & -1 & 1
\end{bmatrix}
$$

## Relaxation Method - Python: Static Eleasticity

> Finite difference method to solve the static problem

If we take the three-point operator  for the second derivative,
we can rewrite the static problem as

$$
    -\mu\nabla^2u = f\rightarrow -\mu\frac{u_{i+1} - 2u_i + u_{i-1}}{h_i^2} = f_i\,.
$$

If we explicit the $u_i$ in our equation, we get

$$
    u_i = \frac{u_{i+1} + u_{i-1}}{2} + \frac{h_i^2}{2\mu}f_i\,,
$$
therefore, the amplitude is given by the average at its surrounding
plus the force contribution evaluated at that point. A way of solving
this is start with a guessing for the initial state and keep updating
it. After a "good amount of iterations", we expect to converge to the
correct result. A way to compare the obtained result is comparing it
to the finite difference approach
> Notebook containing the numerical comparison entitled "finite-element"